# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.90,66,61,10.51,TF,1702835547
1,1,ocean shores,46.9737,-124.1563,9.97,84,49,3.13,US,1702836019
2,2,salekhard,66.5300,66.6019,-21.53,100,100,1.29,RU,1702836020
3,3,waitangi,-43.9535,-176.5597,16.01,94,86,2.24,NZ,1702835549
4,4,glace bay,46.1969,-59.9570,2.83,71,75,7.20,CA,1702836021


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
geo_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City"
)

# Display the map plot
geo_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21, wind speed < 4.5 and cloudiness=0 
city_data_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] <27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0 )].copy()
# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
117,117,nova sintra,14.8667,-24.7167,22.10,67,0,3.70,CV,1702836109
129,129,saint-pierre,-21.3393,55.4781,25.82,69,0,2.57,RE,1702836116
156,156,sultanah,24.4926,39.5857,23.25,11,0,2.57,SA,1702836134
197,197,abu dhabi,24.4667,54.3667,25.53,53,0,2.06,AE,1702836161
282,282,lompoc,34.6391,-120.4579,22.74,46,0,3.09,US,1702836087
360,360,jiwani,25.0500,61.7417,22.53,30,0,4.15,PK,1702836286
564,564,la possession,-20.9289,55.3351,25.99,85,0,3.60,RE,1702836418


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
117,117,nova sintra,14.8667,-24.7167,22.10,67,0,3.70,CV,1702836109,
129,129,saint-pierre,-21.3393,55.4781,25.82,69,0,2.57,RE,1702836116,
156,156,sultanah,24.4926,39.5857,23.25,11,0,2.57,SA,1702836134,
197,197,abu dhabi,24.4667,54.3667,25.53,53,0,2.06,AE,1702836161,
282,282,lompoc,34.6391,-120.4579,22.74,46,0,3.09,US,1702836087,
360,360,jiwani,25.0500,61.7417,22.53,30,0,4.15,PK,1702836286,
564,564,la possession,-20.9289,55.3351,25.99,85,0,3.60,RE,1702836418,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:

# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "lang": "en"    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    lng = hotel_df.loc[index, "Lng"]
    lat = hotel_df.loc[index, "Lat"]
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nova sintra - nearest hotel: Residência Ka Dencho
saint-pierre - nearest hotel: Tropic Hotel
sultanah - nearest hotel: فندق جلنار
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
jiwani - nearest hotel: No hotel found
la possession - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
117,117,nova sintra,14.8667,-24.7167,22.10,67,0,3.70,CV,1702836109,Residência Ka Dencho
129,129,saint-pierre,-21.3393,55.4781,25.82,69,0,2.57,RE,1702836116,Tropic Hotel
156,156,sultanah,24.4926,39.5857,23.25,11,0,2.57,SA,1702836134,فندق جلنار
197,197,abu dhabi,24.4667,54.3667,25.53,53,0,2.06,AE,1702836161,Marriott Downtown Abu-Dhabi
282,282,lompoc,34.6391,-120.4579,22.74,46,0,3.09,US,1702836087,Embassy Suites by Hilton Lompoc Central Coast
360,360,jiwani,25.0500,61.7417,22.53,30,0,4.15,PK,1702836286,No hotel found
564,564,la possession,-20.9289,55.3351,25.99,85,0,3.60,RE,1702836418,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
geo_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
geo_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)